# Merge Two Places
Notebook for merging two places into a single folder.

In [16]:
import os
import json
import pandas as pd

import geopandas as gpd
import glob
with open('../config.json') as f:
    CONFIG = json.load(f)

# Global constants from config
DATA_FOLDER = CONFIG['data_folder']



In [2]:
place_1 = "costa_rica"
place_2 = "panama"

In [4]:
# Creates destination folder
dest_folder = os.path.join(DATA_FOLDER, f"{place_1}-{place_2}")
if not os.path.exists(dest_folder):
    os.mkdir(dest_folder)

In [15]:
# Merges Data Pop Data
data_pop_1 = pd.read_csv(glob.glob(os.path.join(DATA_FOLDER, place_1, "*_pd_2020_1km_ASCII_XYZ.csv"))[0])
data_pop_2 = pd.read_csv(glob.glob(os.path.join(DATA_FOLDER, place_2, "*_pd_2020_1km_ASCII_XYZ.csv"))[0])

data_pop = pd.concat((data_pop_1, data_pop_2), ignore_index=True)
data_pop.to_csv(os.path.join(dest_folder, "pd_2020_1km_ASCII_XYZ.csv"), index=False)


In [20]:
# Merges Populated Places
populated_places_1 = gpd.read_file(glob.glob(os.path.join(DATA_FOLDER, place_1, "hotosm_*_populated_places_points_shp"))[0])
populated_places_2 = gpd.read_file(glob.glob(os.path.join(DATA_FOLDER, place_2, "hotosm_*_populated_places_points_shp"))[0])


populated_places = pd.concat((populated_places_1, populated_places_2), ignore_index=True)
populated_places.to_file(os.path.join(dest_folder, "hotosm_populated_places_points_shp"))


In [21]:
# Merges Building Polygons
building_polygons_1 = gpd.read_file(glob.glob(os.path.join(DATA_FOLDER, place_1, "hotosm_*_buildings_polygons_shp"))[0])
building_polygons_2 = gpd.read_file(glob.glob(os.path.join(DATA_FOLDER, place_2, "hotosm_*_buildings_polygons_shp"))[0])


building_polygons = pd.concat((building_polygons_1, building_polygons_2), ignore_index=True)
building_polygons.to_file(os.path.join(dest_folder, "hotosm_buildings_polygons_shp"))

In [22]:
# Merges Building Polygons
roads_1 = gpd.read_file(glob.glob(os.path.join(DATA_FOLDER, place_1, "hotosm_*_roads_lines_shp"))[0])
roads_2 = gpd.read_file(glob.glob(os.path.join(DATA_FOLDER, place_2, "hotosm_*_roads_lines_shp"))[0])


In [51]:
inter = gpd.sjoin(roads_1[["osm_id","geometry"]], roads_2[["osm_id","geometry"]])

ids_1 = inter.index.values
ids_2 = inter["index_right"].values

temp_roads_1 = roads_1.loc[ids_1].copy()
temp_roads_1["group"] = range(temp_roads_1.shape[0])
temp_roads_2 = roads_2.loc[ids_2].copy()
temp_roads_2["group"] = range(temp_roads_2.shape[0])


temp_roads = pd.concat((temp_roads_1, temp_roads_2), ignore_index=True).sort_values("group")
final_roads = temp_roads.dissolve(by='group').reset_index()[roads_1.columns]

final_roads.to_file("temp")

# Concats the three element
roads = pd.concat((roads_1.loc[~roads_1.index.isin(ids_1)], roads_2.loc[~roads_2.index.isin(ids_2)], final_roads), ignore_index=True)
roads.to_file(os.path.join(dest_folder, "hotosm_roads_lines_shp"))
